# Project Overview

This project demonstrates the reflection pattern within an agentic AI workflow that autonomously generates and improves SQL generation.

The notebook replicates an exercise from Deeplearning.AI’s “Agentic AI” course, adapted to use a different dataset. Majority of the code in this notebook is from the “Agentic AI” course.


# Agentic Workflow
1. LLM: Use LLM to Query a Database
2. Execution Code: Query Exection
3. LLM: Use LLM as Reflection (Give Query Output as Feedback)

# 0. Import Packages, Load Data, and Setup

In [13]:
import json
import utils
import pandas as pd
from dotenv import load_dotenv

_ = load_dotenv()

In [14]:
# ai suite: https://github.com/andrewyng/aisuite
import aisuite as ai

client = ai.Client()

In [25]:
# load utils
import importlib, utils
importlib.reload(utils)

<module 'utils' from '/Users/chohasong/Documents/GitHub/Agentic-AI-Reflection-Design-Pattern-2/utils.py'>

In [18]:
# set up database 
utils.create_transactions_db()

'/Users/chohasong/Documents/GitHub/Agentic-AI-Reflection-Design-Pattern-2/products.db'

In [5]:
utils.print_html(utils.get_schema("products.db"))


cid,name,type,notnull,default,pk
0,id,INTEGER,0,None,1
1,product_id,INTEGER,0,None,0
2,product_name,TEXT,0,None,0
3,brand,TEXT,0,None,0
4,category,TEXT,0,None,0
5,color,TEXT,0,None,0
6,action,TEXT,0,None,0
7,qty_delta,INTEGER,0,None,0
8,unit_price,REAL,0,None,0
9,notes,TEXT,0,None,0


# 1. Build SQL Generator

## 1a. Use an LLM to Query a Database

- Turn NLP query question into a text
- Provide question and the database schema as input
- Example is: "Which color of product has the highest total sales?" given a product schema (product, brand, category, color, action, qty, price, ts)

In [19]:
# Example usage of generate_sql

# We provide the schema as a string
schema = """
Table name: transactions
id (INTEGER)
product_id (INTEGER)
product_name (TEXT)
brand (TEXT)
category (TEXT)
color (TEXT)
action (TEXT)
qty_delta (INTEGER)
unit_price (REAL)
notes (TEXT)
ts (DATETIME)
"""

# We ask a question about the data in natural language
question = "Which color of product has the highest total sales?"

utils.print_html(question, title="User Question")

# Generate the SQL query using the specified model
sql_V1 = utils.generate_sql(question, schema, model="openai:gpt-4.1")

# Display the generated SQL query
utils.print_html(sql_V1, title="SQL Query V1")

## 1b. Query Execution for Query v1 based on LLM v1

In [26]:
# Execute the generated SQL query (sql_V1) against the products.db database.
# The result is returned as a pandas DataFrame.
df_sql_V1 = utils.execute_sql(sql_V1, db_path='products.db')

# Render the DataFrame as an HTML table in the notebook.
# This makes the query output easier to read and interpret.
utils.print_html(df_sql_V1, title="Output of SQL Query V1 - ❌ Does NOT fully answer the question")


**Results**

- The query returns Silver product, with highest total sales of negative value. 

- This is because total sales is determined by qty_delta * unit price. 

- qty_delta is negative for sales (inventory leaving), and positive number for returns or restocks (inventory added)

- The query used SUM(qty_delta), so when summing across transactions, the sales dominate returns and produce a negative total sales.

- The LLM query output is technically valid but semantically wrong because we weren't expecting a negative total sales. This suggests a need for reflection so that we get the total sales.

## 1c. Improving SQL Queries with Reflection

Inputs:

- the user’s question
- the original SQL query
- the table schema

Outputs:

- feedback → a short evaluation (e.g., “valid but missing a date filter”)
- refined_sql → the final SQL (unchanged if correct, or updated if improvements are needed)

In [27]:
def refine_sql(
    question: str,
    sql_query: str,
    schema: str,
    model: str,
) -> tuple[str, str]:
    """
    Reflect on whether a query's *shown output* answers the question,
    and propose an improved SQL if needed.
    Returns (feedback, refined_sql).
    """
    prompt = f"""
You are a SQL reviewer and refiner.

User asked:
{question}

Original SQL:
{sql_query}

Table Schema:
{schema}

Step 1: Briefly evaluate if the SQL OUTPUT fully answers the user's question.
Step 2: If improvement is needed, provide a refined SQL query for SQLite.
If the original SQL is already correct, return it unchanged.

Return STRICT JSON with two fields:
{{
  "feedback": "<1-3 sentences explaining the gap or confirming correctness>",
  "refined_sql": "<final SQL to run>"
}}
"""
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )

    content = response.choices[0].message.content
    try:
        obj = json.loads(content)
        feedback = str(obj.get("feedback", "")).strip()
        refined_sql = str(obj.get("refined_sql", sql_query)).strip()
        if not refined_sql:
            refined_sql = sql_query
    except Exception:
        # Fallback if model doesn't return valid JSON
        feedback = content.strip()
        refined_sql = sql_query

    return feedback, refined_sql


- Display the initial SQL query (V1) generated for the question “Which color of product has the highest total sales?”
- Show the model’s feedback and its refined SQL proposal (V2).
- Execute the original SQL (V1) against the database and present its actual output, so you can see why refinement was needed.

In [28]:
# Example: refine the generated SQL (V1 → V2)

feedback, sql_V2 = refine_sql(
    question=question,
    sql_query=sql_V1,   # <- comes from generate_sql() (V1)
    schema=schema, # <- we reuse the schema from section 3.1
    model="openai:gpt-4.1"
)

# Display the original prompt
utils.print_html(question, title="User Question")

# --- V1 ---
utils.print_html(sql_V1, title="Generated SQL Query (V1)")

# Execute and show V1 output
df_sql_V1 = utils.execute_sql(sql_V1, db_path='products.db')
utils.print_html(df_sql_V1, title="SQL Output of V1 - ❌ Does NOT fully answer the question")

# --- Feedback + V2 ---
utils.print_html(feedback, title="Feedback on V1")
utils.print_html(sql_V2, title="Refined SQL Query (V2)")

# Execute and show V2 output
df_sql_V2 = utils.execute_sql(sql_V2, db_path='products.db')
utils.print_html(df_sql_V2, title="SQL Output of V2 - ❌ Does NOT fully answer the question")

The output still returns incorrect sql query. This happened because the SQL multiplied qty_delta * unit_price without considering that sales events store negative quantities (qty_delta < 0). The sign inversion (+ vs –) is a subtle semantic issue that cannot always be detected by reviewing the query text alone

That’s why the agent must also reflect on the execution output — to catch problems like wrong signs, missing filters, or incorrect aggregations. External feedback from query results grounds the refinement process in reality, not just in the structure of the SQL.

## 1d. Refine an SQL Query with External Feedback

- Display the original SQL query (V1) generated from your question.
- Show the output of V1, which highlights why the initial attempt does not fully answer the question.
- Provide feedback from the LLM based on that output.
- Present the refined SQL query (V2) that addresses the issue.
- Execute V2 and display its output, confirming that it now ✅ fully answers the question.

What’s different?

Signal used

refine_sql: Uses question + schema + original SQL only. No visibility into what V1 returned.

refine_sql_external_feedback: Adds df_feedback (the actual V1 output) converted to Markdown. The model sees the result data and can catch issues like empty rows, wrong grouping, missing filters, wrong metric (e.g., forgot WHERE action='sale'), etc.

Objective

refine_sql: “Improve the query” based on intent and schema reasoning.

refine_sql_external_feedback: Evaluate whether V1 answered the question, then propose a refined SQL if needed. It’s an evaluative loop, not just a rewrite.

In [31]:
def refine_sql_external_feedback(
    question: str,
    sql_query: str,
    df_feedback: pd.DataFrame, ## integrate feedback 
    schema: str,
    model: str,
) -> tuple[str, str]:
    """
    Evaluate whether the SQL result answers the user's question and,
    if necessary, propose a refined version of the query.
    Returns (feedback, refined_sql).
    """
    prompt = f"""
    You are a SQL reviewer and refiner.

    User asked:
    {question}

    Original SQL:
    {sql_query}

    SQL Output:
    {df_feedback.to_markdown(index=False)}

    Table Schema:
    {schema}

    Step 1: Briefly evaluate if the SQL output answers the user's question.
    Step 2: If the SQL could be improved, provide a refined SQL query.
    If the original SQL is already correct, return it unchanged.

    Return a strict JSON object with two fields:
    - "feedback": brief evaluation and suggestions
    - "refined_sql": the final SQL to run
    """

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=1.0,
    )

    
    content = response.choices[0].message.content
    try:
        obj = json.loads(content)
        feedback = str(obj.get("feedback", "")).strip()
        refined_sql = str(obj.get("refined_sql", sql_query)).strip()
        if not refined_sql:
            refined_sql = sql_query
    except Exception:
        # Fallback if the model does not return valid JSON:
        # use the raw content as feedback and keep the original SQL
        feedback = content.strip()
        refined_sql = sql_query

    return feedback, refined_sql

In [35]:
# Output of V1
df_sql_V1

,color,total_sales
0,Silver,-23348.03


In [34]:
# Example: Refine SQL with External Feedback (V1 → V2)

# Execute the original SQL (V1)
df_sql_V1 = utils.execute_sql(sql_V1, db_path='products.db')

# Use external feedback to evaluate and refine
feedback, sql_V2 = refine_sql_external_feedback(
    question=question,
    sql_query=sql_V1,   # V1 query
    df_feedback=df_sql_V1,    # Output of V1
    schema=schema,
    model="openai:gpt-4.1"
)

# --- V1 ---
utils.print_html(question, title="User Question")
utils.print_html(sql_V1, title="Generated SQL Query (V1)")
utils.print_html(df_sql_V1, title="SQL Output of V1 - ❌ Does NOT fully answer the question")

# --- Feedback & V2 ---
utils.print_html(feedback, title="Feedback on V1")
utils.print_html(sql_V2, title="Refined SQL Query (V2)")

# Execute and display V2 results
df_sql_V2 = utils.execute_sql(sql_V2, db_path='products.db')
utils.print_html(df_sql_V2, title="SQL Output of V2 (with External Feedback) - ✅ Fully answers the question")
